In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from skimage import measure
from shapely.geometry.polygon import Polygon
import matplotlib.pyplot as plt
import folium

In [2]:
path_data = "/glade/p/cisl/aiml/jtti_tornado/wofs_preds/wofs_stitched_light.nc"
path_save = "/glade/p/cisl/aiml/jtti_tornado/unet_contours/"

In [3]:
ds = xr.open_dataset(path_data)
lons = ds['lon'] - 360
lats = ds['lat']
ds

<xarray.Dataset>
Dimensions:            (lat: 396, lon: 508, time: 73)
Coordinates:
  * time               (time) datetime64[ns] 2019-05-01 ... 2019-05-01T06:00:00
  * lon                (lon) float64 258.0 258.0 258.0 ... 268.5 268.5 268.6
  * lat                (lat) float64 31.0 31.02 31.04 ... 39.19 39.21 39.23
Data variables:
    UH                 (time, lat, lon) float64 ...
    ZH_1km             (time, lat, lon) float64 ...
    predicted_tor      (time, lat, lon) float64 ...
    predicted_sig_tor  (time, lat, lon) float64 ...

In [4]:
df = pd.DataFrame()
for time in ds["time"].values:
    uh = ds.sel(time=time).to_dataframe()['UH'].unstack('lat').values
    for intensity in [25, 50, 75, 100, 125, 150]:
        contours = measure.find_contours(uh, intensity)
        for i,contour in enumerate(contours):
            if contour.shape[0] < 3:
                continue
            else:
                contour = np.round(contour).astype(int)
                lons_contour = lons[contour[:, 0:1].flatten()].values.reshape(-1,1)
                lats_contour = lats[contour[:, 1:2].flatten()].values.reshape(-1,1)
                contour = np.hstack((lons_contour, lats_contour))
                contour_dict = {"intensity": intensity, "time":time, "geometry": Polygon(contour)}
                df = df.append(contour_dict, ignore_index=True)
gdf_uh = gpd.GeoDataFrame(df, geometry="geometry")
gdf_uh = gdf_uh.set_crs("EPSG:4326")
gdf_uh.to_file(os.path.join(path_save, 'unet_uh.geojson'), driver='GeoJSON')

In [5]:
style1 = {'fillColor': '#4E79A7', 'color': '#4E79A7'}
style2 = {'fillColor': '#A0CBE8', 'color': '#A0CBE8'}
style3 = {'fillColor': '#F28E2B', 'color': '#F28E2B'}
style4 = {'fillColor': '#FFBE7D', 'color': '#FFBE7D'}
style5 = {'fillColor': '#59A14F', 'color': '#59A14F'}
style6 = {'fillColor': '#8CD18D', 'color': '#8CD18D'}
style7 = {'fillColor': '#B6992D', 'color': '#B6992D'}
style8 = {'fillColor': '#F1CE63', 'color': '#F1CE63'}
style9 = {'fillColor': '#499894', 'color': '#499894'}
style10 = {'fillColor': '#86BCB6', 'color': '#86BCB6'}
style11 = {'fillColor': '#E15759', 'color': '#E15759'}
style12 = {'fillColor': '#FF9D9A', 'color': '#FF9D9A'}
style13 = {'fillColor': '#79706E', 'color': '#79706E'}
style14 = {'fillColor': '#BAB0AC', 'color': '#BAB0AC'}
style15 = {'fillColor': '#D37295', 'color': '#D37295'}
style16 = {'fillColor': '#FABFD2', 'color': '#FABFD2'}
style17 = {'fillColor': '#B07AA1', 'color': '#B07AA1'}
style18 = {'fillColor': '#D4A6C8', 'color': '#D4A6C8'}
style19 = {'fillColor': '#9D7660', 'color': '#9D7660'}
style20 = {'fillColor': '#D7B5A6', 'color': '#D7B5A6'}

In [6]:
m = folium.Map(location=[40, -90], zoom_start=5, tiles='CartoDB positron')
folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==25]['geometry'], style_function=lambda x:style1).add_to(m)
folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==50]['geometry'], style_function=lambda x:style2).add_to(m)
folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==75]['geometry'], style_function=lambda x:style3).add_to(m)
folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==100]['geometry'], style_function=lambda x:style4).add_to(m)
folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==125]['geometry'], style_function=lambda x:style5).add_to(m)
# for i, intensity in enumerate([25, 50, 75, 100, 125, 150][:5]):
#     style=styles[i]
#     folium.GeoJson(data=gdf_uh[gdf_uh['intensity']==intensity]['geometry'], style_function=lambda x:styles[i]).add_to(m)
m

In [ ]:
# radar reflectivity dbz
df = pd.DataFrame()
for time in ds["time"].values:
    zh = ds.sel(time=time).to_dataframe()['ZH_1km'].unstack('lat').values
    for intensity in np.arange(10, 85, 5):
        contours = measure.find_contours(zh, intensity)
        for i,contour in enumerate(contours):
            if contour.shape[0] < 3:
                continue
            else:
                contour = np.round(contour).astype(int)
                lons_contour = lons[contour[:, 0:1].flatten()].values.reshape(-1,1)
                lats_contour = lats[contour[:, 1:2].flatten()].values.reshape(-1,1)
                contour = np.hstack((lons_contour, lats_contour))
                contour_dict = {"intensity": intensity, "time":time, "geometry": Polygon(contour)}
                df = df.append(contour_dict, ignore_index=True)
gdf_zh = gpd.GeoDataFrame(df, geometry="geometry")
gdf_zh = gdf_zh.set_crs("EPSG:4326")
gdf_zh.to_file(os.path.join(path_save, 'unet_zh.geojson'), driver='GeoJSON')

In [ ]:
style1 = {'fillColor': '#003f5c', 'color': '#003f5c'}
m = folium.Map(location=[40, -90], zoom_start=5, tiles='CartoDB positron')
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==10]['geometry'], style_function=lambda x:style1).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==15]['geometry'], style_function=lambda x:style2).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==20]['geometry'], style_function=lambda x:style3).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==25]['geometry'], style_function=lambda x:style4).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==30]['geometry'], style_function=lambda x:style5).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==35]['geometry'], style_function=lambda x:style6).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==40]['geometry'], style_function=lambda x:style7).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==45]['geometry'], style_function=lambda x:style8).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==50]['geometry'], style_function=lambda x:style9).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==55]['geometry'], style_function=lambda x:style10).add_to(m)
folium.GeoJson(data=gdf_zh[gdf_zh['intensity']==60]['geometry'], style_function=lambda x:style11).add_to(m)
m

In [ ]:
# add predicted_tor and predicted_sig_tor together
minst = 1000000
maxst = 0
df = pd.DataFrame()
for time in ds["time"].values:
    st = ds.sel(time=time).to_dataframe()['predicted_sig_tor'].unstack('lat').values
    st += ds.sel(time=time).to_dataframe()['predicted_tor'].unstack('lat').values
    minst = min(minst, st.min())
    maxst = max(minst, st.max())
    for intensity in np.arange(.10, .70, .10):
        contours = measure.find_contours(st, intensity)
        for i,contour in enumerate(contours):
            if contour.shape[0] < 3:
                continue
            else:
                contour = np.round(contour).astype(int)
                lons_contour = lons[contour[:, 0:1].flatten()].values.reshape(-1,1)
                lats_contour = lats[contour[:, 1:2].flatten()].values.reshape(-1,1)
                contour = np.hstack((lons_contour, lats_contour))
                contour_dict = {"intensity": intensity, "time":time, "geometry": Polygon(contour)}
                df = df.append(contour_dict, ignore_index=True)
gdf_st = gpd.GeoDataFrame(df, geometry="geometry")
gdf_st = gdf_st.set_crs("EPSG:4326")
gdf_st.to_file(os.path.join(path_save, 'unet_st.geojson'), driver='GeoJSON')

In [ ]:
style1 = {'fillColor': '#003f5c', 'color': '#003f5c'}
m = folium.Map(location=[40, -90], zoom_start=5, tiles='CartoDB positron')
folium.GeoJson(data=gdf_st[gdf_st['intensity']==0.1]['geometry'], style_function=lambda x:style1).add_to(m)
folium.GeoJson(data=gdf_st[gdf_st['intensity']==0.2]['geometry'], style_function=lambda x:style2).add_to(m)
m